**Extracción de Tablas de Premios Oscar (link) con BeautifulSoup:**
- Fecha de la ceremonia.
- Mejor película.
- Mejor director.
- Mejor actor.
- Mejor actriz.
**NOTA:** La información deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. 

In [1]:
# Importamos librerias
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
# Definir la url
url_oscars = "https://en.wikipedia.org/wiki/Academy_Awards"

# Hacer la llamada a la url
res_oscars = requests.get(url_oscars)

# Comprobar el codigo de respuesta
res_oscars.status_code
print(f"La respuesta de la llamada es: {res_oscars.status_code}")


In [ ]:
# Mostrar contenido
res_oscars.content

In [ ]:
# Usar BeautifulSoup
sopa_oscars = BeautifulSoup(res_oscars.content, "html.parser")
print(sopa_oscars.prettify())